In [ ]:
%load_ext autoreload
%autoreload 2
# %matplotlib widget

Requirements
  1. numpy
  2. multiprocessing


## <span style="color:#44FF44;">Using multiprocessing.pool</span>
1. pool.map

In [27]:
! python 00_tuto.py 1
! python 00_tuto.py 0


['00_tuto.py', '1'] True
With multiprocessing
The square of 1 is 1. PID: 35168
The square of 2 is 4. PID: 26392
The square of 3 is 9. PID: 29128
The square of 4 is 16. PID: 6944
The square of 5 is 25. PID: 35232
The square of 6 is 36. PID: 27876
The square of 7 is 49. PID: 12296
The square of 8 is 64. PID: 35608
The square of 9 is 81. PID: 32352
The square of 10 is 100. PID: 37940
Total execution time:  1.377102
['00_tuto.py', '0'] False
No multiprocessing
The square of 1 is 1. PID: 44412
The square of 2 is 4. PID: 44412
The square of 3 is 9. PID: 44412
The square of 4 is 16. PID: 44412
The square of 5 is 25. PID: 44412
The square of 6 is 36. PID: 44412
The square of 7 is 49. PID: 44412
The square of 8 is 64. PID: 44412
The square of 9 is 81. PID: 44412
The square of 10 is 100. PID: 44412
Total execution time:  10.004394


2. Job numbers and reducing execution time

In [28]:
! python 01_tuto.py 10
! python 01_tuto.py 14
! python 01_tuto.py 16
! python 01_tuto.py 20
! python 01_tuto.py 25

Total execution time:  2.28
Total execution time:  2.33
Total execution time:  2.31
Total execution time:  1.49
Total execution time:  1.58


3. Using `pool.starmap`
   
For more than 1 argument

In [29]:
! python 02_tuto.py 10

Argument list number of Jobs X length of argument list
[[9 1]
 [2 4]
 [8 0]]
08x00=00| PID: 45952
02x04=08| PID: 2780009x01=09| PID: 45788

Total execution time:  1.28


4. Using `pool.apply_async`

More control over the multiprocessing tasks <br>
Assigning multiple functions in a session <br>
Releassing the parent process for other tasksto do <br>

In [30]:
! python 03_tuto.py

Process starting time:  0.00
PID 30336: 2 * 3 = 6
PID 32532: 4 * 5 = 20
PID 27088: 6 * 7 = 42
PID 36508: 0 * 1 = 0
Total execution time:  1.18
Final results: [6, 20, 42, 0]


![Table of pool methods](poolTable.png)

## <span style="color:#FFAA00;">Using multiprocessing.Process</span>
Let's leave this for children to play with and focus more on cool stuff<br>
Also, multiprocessing.Process is friendly with notebooks but... 


In [37]:
import multiprocessing

# Define a function to be executed by the process
def worker_function(name):
    print(f"Hello, {name}!")

if __name__=='__main__':
# if True:
  # Create a multiprocessing.Process instance
  process1 = multiprocessing.Process(target=worker_function, args=("Charlie the Harley"  ,))
  process2 = multiprocessing.Process(target=worker_function, args=("Sophie the trophy"   ,))
  process3 = multiprocessing.Process(target=worker_function, args=("Albert the shepherd" ,))
  
  # Start the process
  process1.start()
  process2.start()
  process3.start()

  # Wait for the process to finish (optional)
  print("Main program continues...")

  process1.join()
  process2.join()
  process3.join()

Main program continues...


In [34]:
! python 04_tuto.py

Hello, Charlie the Harley!
Hello, Sophie the trophy!
Hello, Albert the shepherd!
Main program continues...


In the notebook: 
1. The kernel will kill itself (suicide) if any cell ended running and a process is alive. join all processes before ending a cell
2. If `Process` is called directly inside the notebook, the children can't reach the casting interface (can't `print()`) but it's possible if the Process is inside a module/script that have been imported by the notebook.
3. In any python script, `Process` and `Pool` should be called in main scope (`if __name__=='__main__':...`) because the script is called inside all processes.
4. We can use args or kwargs arguments in `Process` to pass arguments or keyworded arguments.
5. There is no number of processes to specify in this case so it should be manually done 

In [38]:
! python 05_tuto.py

PID  39136
PID  5052
PID  5688
PID  44832
PID  28480
PID  28772
PID  18868
PID  14184
PID  39988
PID  7276
PID  39372
PID  31196
PID  30752
PID  10224
PID  14124
PID  5532
PID  24444
PID  21844
PID  26292
PID  20284
Total time 2.476343 seconds


`for ind,p in enumerate(processes): `<br>  
&emsp;`if not p.is_alive():`<br>
&emsp;&emsp;`p.close()`<br>
&emsp;&emsp;`processes.pop(p)`<br>

It's to close all the processes that have ended their jobs. This is important if you have a lot of processes >100s. There is some files created and opened and all systems have a limited amount of memory for opening files simultaneously.

## <span style="color:#FF5500;">Sharing  memory</span>

In [40]:
from multiprocessing import shared_memory
import numpy as np
# Creating a parent matrix
matrix = np.random.randint(0,10,size=(3,))

shm_a = shared_memory.SharedMemory(create=True, size=matrix.nbytes)
shm_matrix= np.ndarray(shape=matrix.shape,dtype=matrix.dtype, buffer=shm_a.buf)
shm_matrix[:] =matrix

It's important to fill the shared matrix using slices (`[:]`) else equalizing two matrices directly will break the link and delete the buffer and build a new variable

In [41]:
shm_matrix2 = np.ndarray(shape=matrix.shape,dtype=matrix.dtype, buffer=shm_a.buf)
shm_matrix3 = np.ndarray(shape=matrix.shape,dtype=matrix.dtype, buffer=shm_a.buf)
print(shm_matrix2,shm_matrix3)

[8 1 7] [8 1 7]


In [42]:
shm_matrix3[0] = 100
print(shm_matrix2,shm_matrix3)

[100   1   7] [100   1   7]


In [45]:
!python 07_tuto.py 1 10

[5 1 4 2 7 7 5 9 8 5]
[5 1 4 4 7 7 5 9 8 5]
[5 2 4 4 7 7 5 9 8 5]
[5 9 4 4 7 7 5 9 8 5]
[5 9 7 7 7 7 5 9 8 5]
[5 9 7 7 7 7 0 9 8 9]
[5 9 7 7 7 7 0 9 8 9]
[5 9 7 7 7 7 0 9 8 9]
[9 9 7 7 7 7 0 9 8 9]
[9 9 7 7 7 7 0 9 7 9]
[9 9 2 4 5 4 0 9 5 9]
[9 9 2 4 5 4 0 9 5 9]
[9 9 2 4 5 4 0 9 5 9]
[9 9 9 4 1 4 0 9 3 9]
[9 9 9 8 7 6 0 9 3 9][9 9 9 8 7 6 0 9 3 9]

[9 9 9 8 7 6 0 9 3 9]
[9 9 9 0 7 6 0 9 2 9]
[9 9 9 0 7 6 0 9 2 9]
[9 9 9 0 2 3 0 9 2 9]
[9 9 9 0 8 3 0 9 8 9]
[9 9 9 0 8 3 0 9 8 9]
[9 9 9 0 8 6 0 9 8 9]
[9 9 9 0 1 0 0 9 4 9]
[9 9 9 0 0 0 0 9 8 9]
[9 9 9 0 0 0 0 9 8 9]
[9 9 9 0 0 0 0 9 8 9]
[9 9 9 0 0 0 0 9 1 9]
[9 9 9 0 0 0 0 9 9 9][9 9 9 0 0 0 0 9 9 9]
[9 9 9 0 0 0 0 9 9 9]

[0 6 9 0 0 0 0 8 8 0][0 6 9 0 0 0 0 8 8 0]

[0 6 9 0 0 0 0 8 8 0]
[0 2 9 0 0 0 0 0 1 0][0 2 9 0 0 0 0 0 1 0]

[0 2 9 0 0 0 0 0 1 0]
[0 5 9 0 0 0 0 0 1 0]
[0 5 7 0 0 0 0 0 1 0]
[0 5 7 0 0 0 0 0 1 0]
[0 3 9 0 0 0 0 0 7 0]
[0 3 9 0 0 0 0 0 7 0]
[0 6 9 0 0 0 0 0 7 0]
[0 6 0 0 0 0 0 0 7 0]
[0 6 0 0 0 0 0 0 3 0]
[0 0 0 0 0

### The small cherry on the top

In [ ]:
from multiprocessing.shared_memory import SharedMemory
from time import sleep
shape =(3,) 
dtype = int 
shmm_name = "wnsm_1201d560"

shmm = SharedMemory(create=False,name=shmm_name)
data = np.ndarray(shape,dtype,shmm.buf)
while True:
  print(data)
  sleep(0.5)


<!-- Concurent.futures -->
